In [6]:
%run common.py

Planning start time: Jul 13, 10:00 (1468393200)


In [7]:
'''
    Examples:
    al = pd.read_csv(FOLDER + '/mandatory/travel_times_all_pairs.csv', sep=';')
    get_longest_pair(['МАРИИНСК', 'ИЛАНСКАЯ', 'ИРКУТСК-СОРТИРОВОЧНЫЙ', 'РЕШОТЫ'], al.set_index(['st_from_name', 'st_to_name']))
    
    => Out[460]: ('МАРИИНСК', 'ИРКУТСК-СОРТИРОВОЧНЫЙ', 94170)
    
    get_longest_pair(['МАРИИНСК', 'ИЛАНСКАЯ', 'РЕШОТЫ', 'ИРКУТСК-СОРТИРОВОЧНЫЙ'], nx.all_pairs_dijkstra_path_length(g))
    
    => Out[460]: ('МАРИИНСК', 'ИРКУТСК-СОРТИРОВОЧНЫЙ', 94170)    
'''

def get_longest_pair(st_list, lengths):
    sm1, sm2 = '', ''
    m = 0
    for s1 in st_list:
        for s2 in st_list:
            if type(lengths) == dict:
                l = lengths[s1][s2]
            elif type(lengths) == pd.DataFrame:
                l = lengths.ix[s1, s2].values[0]
            else: l = 0
            if l > m:
                m = l
                sm1, sm2 = s1, s2
    return (sm1, sm2, m)

In [8]:
all_lengths = pd.read_csv(FOLDER + '/mandatory/travel_times_all_pairs.csv', sep=';').set_index(['st_from_name', 'st_to_name'])
    
def get_reg_name(l):
    l_big = [st for st in l if st in big_st]
    if len(l_big) == 2:
        ret = l_big
    elif len(l_big) > 2:
        st1, st2, length = get_longest_pair(l_big, all_lengths)
        ret = [st1, st2]
    else:
        st1, st2, length = get_longest_pair(l, all_lengths)
        ret = [st1, st2]
    return ret[0] + ' - ' + ret[1]        

team_region = pd.read_csv(FOLDER + 'team_region.csv', dtype={'st_from':str, 'st_to':str, 'depot':str})
add_info(team_region)
big_st = stations[stations.norm_time > 0].name.unique()
team_region['depot_name'] = team_region.depot.map(st_names.name)
team_region['reg_name'] = team_region.team_region\
                            .map(team_region.groupby('team_region').st_from_name.unique().apply(get_reg_name))

cols_tracks = ['team_region', 'asoup', 'depot', 'depot_name', 'st_from_name', 'st_to_name', 'reg_name']
cols_times = ['team_region', 'asoup', 'depot', 'depot_name', 'time_f', 'time_b', 'time_wr']

In [9]:
#print(nice_time(current_time + 20*60 + 3*3600))
nct = current_time + 20*60 + 3*3600
st1 = 'ИРКУТСК-СОРТИРОВОЧНЫЙ'
start_st = team_plan[team_plan.state.isin([0, 1])].drop_duplicates('team').set_index('team')
team_plan['team_type'] = team_plan.team.apply(lambda x: int(str(x)[0]))
team_plan['start_st'] = team_plan.team.map(start_st.st_from)
team_plan.depot.fillna(team_plan.start_st, inplace=True)
team_plan['depot_name'] = team_plan.depot.map(st_names.name)
cols = ['team', 'st_from_name', 'st_to_name', 'time_start_norm', 'time_end_norm', 'state', 'loco']
a = team_plan[(team_plan.time_start >= nct) & (team_plan.time_start < nct + 24 * 3600)
         & (team_plan.state.isin([0, 1])) & (team_plan.st_from_name == st1) & (team_plan.depot_name == st1)]
print(a.st_to_name.value_counts())
print(a.state.value_counts())
print(a.team_type.value_counts())

ГОНЧАРОВО          28
БАТАРЕЙНАЯ         13
ВОЕННЫЙ ГОРОДОК     3
Name: st_to_name, dtype: int64
1    28
0    16
Name: state, dtype: int64
2    44
Name: team_type, dtype: int64


In [10]:
print(nice_time(current_time))
cols = ['team', 'st_from_name', 'st_to_name', 'oper_time_f', 'time_start_norm', 'state', 'wait_ct', 'wait']
team_plan['sinfo'] = team_plan['state_info']
team_plan['oper_time_f'] = team_plan.oper_time.apply(nice_time)
team_plan['wait_ct'] = np.round((current_time - team_plan.oper_time) / 3600, 2)
team_plan['wait'] = np.round((team_plan.time_start - team_plan.oper_time) / 3600, 2)
a = team_plan[(team_plan.state_info == '2') 
          & (team_plan.state.isin([0, 1]))][cols].drop_duplicates('team').sort_values('wait', ascending=False)

(a.wait - a.wait_ct).describe()

Jul 13, 10:00


count    124.000000
mean       3.361452
std        2.799301
min        0.000000
25%        1.427500
50%        2.420000
75%        5.025000
max       10.290000
dtype: float64

In [11]:
st = 'ИРКУТСК-СОРТИРОВОЧНЫЙ'

# trains
train_plan['train_type'] = train_plan.train.apply(lambda x: int(str(x)[0]))
trains = train_plan[(train_plan.time_start >= current_time) & (train_plan.time_start < current_time + 24 * 3600)
          & (train_plan.train_type.isin([2, 9]))
          & (train_plan.st_from_name == st)]

print('Trains:')
print(trains.st_to_name.value_counts().to_string(), '\n-')
print(trains.train_type.value_counts().to_string())

locos = loco_plan[(loco_plan.time_start >= current_time) & (loco_plan.time_start < current_time + 24 * 3600)          
          & (loco_plan.st_from_name == st)]
print('\nLocos:')
print(locos[locos.state == 1].st_to_name.value_counts().to_string(), '\n-')
print(locos.state.value_counts().to_string())

team_plan['team_type'] = team_plan.team.apply(lambda x: int(str(x)[0]))
team_plan['start_st'] = team_plan.team.map\
            (team_plan[team_plan.state.isin([0, 1])].drop_duplicates('team').set_index('team').st_from)
team_plan.depot.fillna(team_plan.start_st, inplace=True)
team_plan['depot_name'] = team_plan.depot.map(st_names.name)
teams = team_plan[(team_plan.time_start >= current_time) & (team_plan.time_start < current_time + 24 * 3600)          
          & (team_plan.st_from_name == st)]
print('\nTeams:')
print(teams[teams.state == 1].st_to_name.value_counts().to_string(), '\n-')
print(teams[teams.state == 1].team_type.value_counts().to_string(), '\n-')
print(teams[teams.state.isin([0, 1])].state.value_counts().to_string(), '\n-')
print(teams[teams.state.isin([0, 1])].depot_name.value_counts().to_string())

Trains:
ГОНЧАРОВО     65
БАТАРЕЙНАЯ    28 
-
2    84
9     9

Locos:
ГОНЧАРОВО     35
БАТАРЕЙНАЯ    27 
-
1    62
0    37
4     2

Teams:
БАТАРЕЙНАЯ    60
ГОНЧАРОВО     38 
-
2    98 
-
1    98
0    42 
-
ЗИМА                     73
ИРКУТСК-СОРТИРОВОЧНЫЙ    60
СЛЮДЯНКА I                4
УЛАН-УДЭ                  2
НИЖНЕУДИНСК               1


In [95]:
df = pd.read_csv('./input/otsev_uth_detail.csv', encoding='utf-8-sig', sep=';', 
                 dtype={'train_id':str, 'train_index':str, 'loco_id':str, 'team_id':str}, 
                 parse_dates=['time', 'train_time', 'loco_time', 'team_time', 'team_ready_time1', 'team_ready_time2'])                 
print(df.columns)

Index(['uns', 'train_id', 'train_index', 'train_number', 'loco_id',
       'loco_name', 'team_id', 'team_name', 'motorman', 'location_asoup_id',
       ...
       'team_location_asoup_id', 'team_location_name', 'team_type_asoup_id',
       'team_type_name', 'team_time', 'team_ready_time1', 'team_ready_st1',
       'team_ready_time2', 'team_ready_st2', 'team_ready_rest_start_time'],
      dtype='object', length=104)


In [97]:
print(nice_time(current_time))
cols = ['uns', 'team_id', 'out',
        'team_type_name', 'team_time',
                                'team_ready_time1', 'team_ready_time2']
df['delta1'] = df.team_ready_time1 - df.team_time
nice_print(df[df.team_type_asoup_id == 54].sort_values('delta1')[cols].head())

Jul 13, 10:00
       uns       team_id  out        team_type_name           team_time    team_ready_time1    team_ready_time2
 885342674  200200256743  NaN  Начало выходного дня 2016-07-12 07:36:00 2016-07-13 01:44:24 2016-07-11 19:33:00
 885438251  200200220837  NaN  Начало выходного дня 2016-07-12 10:12:00 2016-07-13 04:57:00                 NaT
 885173429  200200252139  NaN  Начало выходного дня 2016-07-12 04:27:00 2016-07-12 23:12:00 2016-07-11 22:40:00
 885111604  200200272972  NaN  Начало выходного дня 2016-07-12 02:13:00 2016-07-12 20:58:12                 NaT
 885207307  200200110159  NaN  Начало выходного дня 2016-07-12 05:19:00 2016-07-13 00:04:12 2016-07-11 22:30:00


In [103]:
team_info['depot_time_f'] = team_info.depot_time.apply(nice_time)
team_info['return_time_f'] = team_info.return_time.apply(nice_time)
nice_print(team_info[team_info.team == '200200256743'][['team', 'oper_time_f', 'loc_name', 'depot_time_f', 'return_time_f']])

         team    oper_time_f         loc_name   depot_time_f  return_time_f
 200200256743  Jul 12, 07:36  СЕВЕРОБАЙКАЛЬСК  Jul 13, 01:44  Jul 11, 19:33


In [102]:
m1 = team_info[(team_info.depot_time == -1) 
          & (team_info.return_time == -1)][['team', 'oper_time_f', 'loc_name', 'depot_time_f', 'return_time_f']]

with pd.option_context('display.max_colwidth', 40):
    nice_print(df[df.team_id.isin(m1.team)][cols])

       uns       team_id  out                           team_type_name           team_time team_ready_time1 team_ready_time2
 880496579  200200149070  NaN                   Отправление пассажиром 2016-07-07 15:29:00              NaT              NaT
 881382559  200200226491  NaN                   Отправление пассажиром 2016-07-08 11:31:00              NaT              NaT
 883039999  200200081132  NaN                   Отправление пассажиром 2016-07-10 02:32:00              NaT              NaT
 882103040  200200241908  NaN      Сдача локомотива на территории депо 2016-07-09 04:38:00              NaT              NaT
 862826457  200200191250  NaN       Явка после отдыха в пункте оборота 2016-06-20 17:10:00              NaT              NaT
 881025325  200200257491  NaN       Явка после отдыха в пункте оборота 2016-07-08 04:00:00              NaT              NaT
 879300393  200200205109    1                   Отправление со станции                 NaT              NaT              NaT
